In [5]:
import torch

# 作为一个演示例子，假设我们想对函数y = 2x ⊤ x关于列向量x求导。首先，我们创建变量x并为其分配一个初始值。
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [7]:
# 在我们计算y关于x的梯度之前，需要一个地方来存储梯度。重要的是，我们不会在每次对一个参数求导时都
# 分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
# 注意，一个标量函数关于向量x的梯度是向量，并且与x具有相同的形状。

x.requires_grad_(True)  # 等价于 `x = torch.arange(4.0, requires_grad=True)`
x.grad  # 默认值是None

In [12]:
# 现在让我们计算y。由于x是一个长度为4的向量，我们得到的y是一个标量。
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [13]:
# x是一个长度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。
# 接下来，通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。
y.backward()
x.grad

tensor([ 0.,  8., 16., 24.])

In [14]:
# 现在计算x的另一个函数

# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [15]:
# 非标量变量的反向传播
# 当y不是一个标量时，向量y关于向量x的导数的最自然解释是一个矩阵。
# 对于高阶和高维的y和x，求导的结果可以是一个高阶张量。

# 然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中），但当调用向量的反向计算时，
# 我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
# 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

# 在 PyTorch 中，backward() 函数用于计算张量的梯度。默认情况下，backward() 操作只适用于标量（即只有一个值的张量），因为它会自动计算该标量相对于网络参数的梯度。
# 然而，当你对非标量张量调用 backward() 时，PyTorch 需要你明确传递一个 gradient 参数，告诉它这个非标量的每个分量应该怎么对参数求导。

# 具体解释：
# 1.标量的情况： 
#   当你对标量调用 backward() 时，默认情况下它假设标量是网络最终的输出，也就是标量相对于自己的梯度为 1。因此，backward() 函数能顺利计算出网络参数的梯度。
# 2.非标量的情况： 
#   当你对非标量（比如一个向量或矩阵）调用 backward() 时，PyTorch 无法自动知道如何计算梯度，因为非标量张量有多个值，它不知道每个值如何对网络参数求导。
#   因此，你需要通过 gradient 参数，提供一个和非标量 y 形状相同的张量，表示这个非标量相对于网络输出的梯度。
# 3.传递梯度参数：
#   y.backward(torch.ones_like(y))：当你传递一个全 1 的张量时，表示对 y 的每个元素都使用相同的权重（即假设它们对网络输出的贡献相等）。这是在你只关心所有偏导数的总和时常用的做法。
#   y.sum().backward()：这和传递一个全 1 的 gradient 类似，因为 y.sum() 是标量，所以它的梯度传播只需要传递 1。

# 什么时候销毁计算图：
# 1.构建计算图的时机：计算图是在前向传播过程中，随着每次对 requires_grad=True 的张量进行操作时即时构建的。
# 2.释放计算图的时机：通常情况下，计算图在反向传播完成后会被销毁，除非你明确要求保留它。


# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [21]:
# 分离计算
# 有时，我们希望将某些计算移动到记录的计算图之外。例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。
# 想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数，并且只考虑到x在y被计算后发挥的作用。
# 这里可以分离y来返回一个新变量u，该变量与y具有相同的值，但丢弃计算图中如何计算y的任何信息。
# 换句话说，梯度不会向后流经u到x。因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理，而不是z=x*x*x关于x的偏导数。
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [22]:
# 由于记录了y的计算结果，我们可以随后在y上调用反向传播，得到y=x*x关于的x的导数，即2*x
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [50]:
# Python控制流的梯度计算
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

# 从正态分布中生成的随机标量（形状为 () 的张量）
a =torch.randn(size=(), requires_grad=True)
print(a)
d =f(a)
# d.backward()
d.backward()
a.grad == d / a

tensor(0.0065, requires_grad=True)


tensor(True)